<a href="https://colab.research.google.com/github/Cynthia-menglu/hello-world/blob/main/M5Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from opentrons import protocol_api

metadata = {'apiLevel': '2.8'}
def run(protocol:protocol_api.ProtocolContext):
#labware
  plate = protocol.load_labware('corning_96_wellplate_360ul_flat', 1)
  tiprack_1 = protocol.load_labware('opentrons_96_tiprack_300ul', 2)
  p300 = protocol.load_instrument('p300_multi_gen2', 'left', tip_racks=[tiprack_1])
  reservoir = protocol.load_labware('usascientific_12_reservoir_22ml', 3)
  #A1 Flourescein
  #A2 PBS

  #commands

  #put 200ul F into first well
  p300.pick_up_tip()
  p300.aspirate(200,reservoir['A1']) #Aspirate Flourescein
  p300.dispense(200, plate['A1'])
  p300.drop_tip()

  #PBS into all wells #range of A2 to A12
  p300.pick_up_tip()
  rows = ['A', 'B', 'C', 'D', 'E', 'F', 'G','H']
  columns = [1,2,3,4,5,6,7,8,9,10,11,12]
  for i in columns[1:]:
      p300.aspirate(100, reservoir['A2'])
      p300.dispense(100, plate[f"{rows[0]}{i}"])
  p300.drop_tip()

  #serially dilute from A1 with fluorescein to A12. 

  for i in columns[0:10]:
      p300.transfer(100, plate.wells(f'A{i}'), plate.wells(f'A{i+1}'), mix_after=(3, 50))


In [ ]:
def get_values(*names):
    import json
    _all_values = json.loads("""{"pipette_type":"p300_multi","dilution_factor":2,"num_of_dilutions":10,"total_mixing_volume":200,"tip_use_strategy":"never"}""")
    return [_all_values[n] for n in names]


metadata = {
    'author': 'Opentrons <protocols@opentrons.com>',
    'source': 'Protocol Library',
    'apiLevel': '2.2'
    }


def run(protocol_context):
    [pipette_type, dilution_factor, num_of_dilutions, total_mixing_volume,
        tip_use_strategy] = get_values(  # noqa: F821
            'pipette_type', 'dilution_factor', 'num_of_dilutions',
            'total_mixing_volume', 'tip_use_strategy'
        )

    # labware
    trough = protocol_context.load_labware(
        'usascientific_12_reservoir_22ml', '2')
    liquid_trash = trough.wells()[0]
    plate = protocol_context.load_labware(
        'corning_96_wellplate_360ul_flat', '3')
    tiprack = [
        protocol_context.load_labware('opentrons_96_tiprack_300ul', slot)
        for slot in ['1', '4']
    ]

    pip_name = pipette_type.split('_')[-1]

    pipette = protocol_context.load_instrument(
        pipette_type, mount='left', tip_racks=tiprack)

    transfer_volume = total_mixing_volume/dilution_factor
    diluent_volume = total_mixing_volume - transfer_volume

    if pip_name == 'multi':
        pipette.pick_up_tip()
        pipette.aspirate(200,trough.wells()[1]) #Aspirate Flourescein
        pipette.dispense(200, plate.rows()[0][0])
        pipette.drop_tip()
        # Distribute diluent across the plate to the the number of samples
        # And add diluent to one column after the number of samples for a blank
        pipette.transfer(
            diluent_volume,
            trough.wells()[0],
            plate.rows()[0][1:2+num_of_dilutions]
        )

        # Dilution of samples across the 96-well flat bottom plate
        if tip_use_strategy == 'never':
            pipette.pick_up_tip()

        for s, d in zip(
                plate.rows()[0][:num_of_dilutions],
                plate.rows()[0][1:1+num_of_dilutions]
        ):
            pipette.transfer(
                transfer_volume,
                s,
                d,
                mix_after=(3, total_mixing_volume/2),
                new_tip=tip_use_strategy
            )

        # Remove transfer volume from the last column of the dilution
        pipette.transfer(
            transfer_volume,
            plate.rows()[0][num_of_dilutions],
            liquid_trash,
            new_tip=tip_use_strategy,
            blow_out=True
        )

        if tip_use_strategy == 'never':
            pipette.drop_tip()

    else:
        # Distribute diluent across the plate to the the number of samples
        # And add diluent to one column after the number of samples for a blank
        for col in plate.columns()[1:1+num_of_dilutions]:
            pipette.distribute(
                diluent_volume, trough.wells()[0], [well for well in col])

        for row in plate.rows():
            if tip_use_strategy == 'never':
                pipette.pick_up_tip()

            for s, d in zip(row[:num_of_dilutions], row[1:1+num_of_dilutions]):

                pipette.transfer(
                    transfer_volume,
                    s,
                    d,
                    mix_after=(3, total_mixing_volume/2),
                    new_tip=tip_use_strategy
                )

                pipette.transfer(
                    transfer_volume,
                    row[num_of_dilutions],
                    liquid_trash,
                    new_tip=tip_use_strategy,
                    blow_out=True
                )

            if tip_use_strategy == 'never':
                pipette.drop_tip()


In [ ]:
import pandas
import sys
#!{sys.executable} -m pip install opentrons
import opentrons
print("Omicron")
"""
print(help("modules"))
import os
os.system('pip3 install opentrons')
"""
from opentrons import simulate
metadata = {'apiLevel': '2.8'}
protocol = simulate.get_protocol_api('2.8')
#Labware
plate = protocol.load_labware('corning_96_wellplate_360ul_flat', 1)
tiprack_1 = protocol.load_labware('opentrons_96_tiprack_300ul', 2)
#pipettes
p300 = protocol.load_instrument('p300_single_gen2', 'right', tip_racks=[tiprack_1])
protocol.max_speeds['Z'] = 10
#complex commands
p300.transfer(100, plate['A1'], plate['B1'], mix_before=(2,50), touch_tip=True, blow_out=True, blowout_location='destination well', new_tip='always') 
#block commands
p300.pick_up_tip()
p300.aspirate(100, plate['A2'])
p300.dispense(100, plate['B2'])
#mix(repetitions, volume, location, rate)
p300.mix(3, 50, plate['B2'], 0.5)
p300.blow_out(plate['B2'].bottom(10))
p300.return_tip()
#print out commands so we can check the simulation
for line in protocol.commands(): 
        print(line)




In [ ]:
from opentrons import simulate

metadata = {'apiLevel': '2.7'}
protocol = simulate.get_protocol_api('2.7')


plate = protocol.load_labware('corning_96_wellplate_360ul_flat', 1)
tiprack_1 = protocol.load_labware('opentrons_96_tiprack_300ul', 2)
p300 = protocol.load_instrument('p300_single_gen2', 'right', tip_racks=[tiprack_1])
reservoir = protocol.load_labware('usascientific_12_reservoir_22ml', 3)

for i in range(8):
    p300.distribute(50, reservoir.wells('A1'), plate.rows()[i])

for line in protocol.commands(): 
        print(line)


/root/.opentrons/robot_settings.json not found. Loading defaults
/root/.opentrons/deck_calibration.json not found. Loading defaults


Distributing 50.0 from A1 of USA Scientific 12 Well Reservoir 22 mL on 3 to A1 of Corning 96 Well Plate 360 µL Flat on 1
Transferring 50.0 from A1 of USA Scientific 12 Well Reservoir 22 mL on 3 to A1 of Corning 96 Well Plate 360 µL Flat on 1
Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 2
Aspirating 270.0 uL from A1 of USA Scientific 12 Well Reservoir 22 mL on 3 at 92.86 uL/sec
Dispensing 50.0 uL into A1 of Corning 96 Well Plate 360 µL Flat on 1 at 92.86 uL/sec
Dispensing 50.0 uL into A2 of Corning 96 Well Plate 360 µL Flat on 1 at 92.86 uL/sec
Dispensing 50.0 uL into A3 of Corning 96 Well Plate 360 µL Flat on 1 at 92.86 uL/sec
Dispensing 50.0 uL into A4 of Corning 96 Well Plate 360 µL Flat on 1 at 92.86 uL/sec
Dispensing 50.0 uL into A5 of Corning 96 Well Plate 360 µL Flat on 1 at 92.86 uL/sec
Blowing out at A1 of Opentrons Fixed Trash on 12
Aspirating 270.0 uL from A1 of USA Scientific 12 Well Reservoir 22 mL on 3 at 92.86 uL/sec
Dispensing 50.0 uL into A6 of Corning 96 W

In [ ]:
# Lucy's code: 
#set up module
import sys
!{sys.executable} -m pip install opentrons
from opentrons import simulate
metadata = {'apiLevel': '2.8'}
protocol = simulate.get_protocol_api('2.8')

#set up labware
plate = protocol.load_labware('corning_96_wellplate_360ul_flat', 1)
tiprack_1 = protocol.load_labware('opentrons_96_tiprack_300ul', 2)
p300 = protocol.load_instrument('p300_single_gen2', 'right', tip_racks=[tiprack_1])
reservoir = protocol.load_labware('usascientific_12_reservoir_22ml', 4)
protocol.max_speeds['Z'] = 10 # speed of pipetting (z axis) is 10
# loop through each column of the first four rows
for i in range(4):
    # source well = reservoir
    rows = plate.rows()[i]  # list contains all cells in rows A to D
    p300.distribute(200, reservoir.wells('A1'), rows[0])  # transfer 200uL of reference working solution to A1, B1, C1, D1
    
    # Add 100uL PBS diluent to columns 2-12 for A-D. 
    for r in rows[1:]: # all cells in each row A-D except first
        p300.distribute(100, reservoir.wells('A12'), r)

    p300.transfer(100, rows[:10], rows[1:11], mix_after=(3, 50), touch_tip=True, blow_out=True, blowout_location='destination well', new_tip='always')
    #from each cell 1-10 in each row to each consecutive cell, except 12
    #so from A1-A10, transfer into A2-A11, ignore 12.
    #indexing weird, single indexing of list uses index as initialised at 0, but slicing list uses index as initialised at 1, needs double-checking
    p300.pick_up_tip()
    p300.aspirate(100, rows[10]) # take up volume from position 11 in row
    p300.drop_tip()  # discard into trash 
    
    
for line in protocol.commands(): 
        print(line)

     |████████████████████████████████| 1.0 MB 14.4 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 386 kB 53.9 MB/s 
     |████████████████████████████████| 90 kB 8.3 MB/s 
     |████████████████████████████████| 10.1 MB 68.5 MB/s 
     |████████████████████████████████| 77 kB 4.6 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: jsonschema
    Found existing installation: jsonschema 2.6.0
    Uninstalling jsonschema-2.6.0:
      Successfully uninstalled jsonschema-2.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nbclient 0.5.9 requires jupyter-client>=6.1.5, but you have jupyter-client 5.3.5 which is incompatible.


/root/.opentrons/robot_settings.json not found. Loading defaults
/root/.opentrons/deck_calibration.json not found. Loading defaults


Distributing 200.0 from A1 of USA Scientific 12 Well Reservoir 22 mL on 4 to A1 of Corning 96 Well Plate 360 µL Flat on 1
Transferring 200.0 from A1 of USA Scientific 12 Well Reservoir 22 mL on 4 to A1 of Corning 96 Well Plate 360 µL Flat on 1
Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 2
Aspirating 220.0 uL from A1 of USA Scientific 12 Well Reservoir 22 mL on 4 at 92.86 uL/sec
Dispensing 200.0 uL into A1 of Corning 96 Well Plate 360 µL Flat on 1 at 92.86 uL/sec
Blowing out at A1 of Opentrons Fixed Trash on 12
Dropping tip into A1 of Opentrons Fixed Trash on 12
Distributing 100.0 from A12 of USA Scientific 12 Well Reservoir 22 mL on 4 to A2 of Corning 96 Well Plate 360 µL Flat on 1
Transferring 100.0 from A12 of USA Scientific 12 Well Reservoir 22 mL on 4 to A2 of Corning 96 Well Plate 360 µL Flat on 1
Picking up tip from B1 of Opentrons 96 Tip Rack 300 µL on 2
Aspirating 120.0 uL from A12 of USA Scientific 12 Well Reservoir 22 mL on 4 at 92.86 uL/sec
Dispensing 100.0 uL 